In [76]:
                             
import os
import pandas as pd
from sqlalchemy import create_engine, text
                                      
from sqlalchemy import Integer, String, Text, TIMESTAMP, Boolean, VARCHAR, Float, BigInteger                     
from dotenv import load_dotenv
import logging
import numpy as np
import warnings
import time                    
from IPython.display import display

In [77]:
                                
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

In [78]:
                                                           
logging.info("Cargando variables de entorno...")
dotenv_path = '/home/nicolas/Escritorio/workshops/workshop_2/env/.env'                         
load_dotenv(dotenv_path=dotenv_path)

                                       
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_HOST = os.getenv('POSTGRES_HOST')
DB_PORT = os.getenv('POSTGRES_PORT')
DB_NAME = os.getenv('POSTGRES_DB')                          

                                    
SOURCE_CSV_PATH = '/home/nicolas/Escritorio/workshops/workshop_2/data/youtube_stats.csv'                                           
CLEAN_TABLE_NAME = 'youtube_stats_clean'                                           
CHUNK_SIZE = 40000                                                           

                           
if not all([DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME]):
    logging.error("Faltan variables de entorno para la base de datos en " + dotenv_path)
    raise ValueError("Variables de entorno de DB incompletas.")

logging.info("Variables de entorno cargadas.")

2025-04-11 12:12:10,228 - INFO - Cargando variables de entorno...
2025-04-11 12:12:10,237 - INFO - Variables de entorno cargadas.


In [79]:
                                                     
df_yt_stats = None
logging.info(f"Cargando datos de estadísticas de YouTube desde: {SOURCE_CSV_PATH}")
try:
    df_yt_stats = pd.read_csv(SOURCE_CSV_PATH)
    logging.info(f"Datos cargados exitosamente desde CSV. Filas iniciales: {len(df_yt_stats)}")
except FileNotFoundError:
    logging.error(f"Error: No se encontró el archivo CSV en {SOURCE_CSV_PATH}. Asegúrate de que el script anterior lo haya generado.")
    raise
except Exception as e:
    logging.error(f"Error al leer el archivo CSV {SOURCE_CSV_PATH}: {e}")
    raise

2025-04-11 12:12:10,254 - INFO - Cargando datos de estadísticas de YouTube desde: /home/nicolas/Escritorio/workshops/workshop_2/data/youtube_stats.csv
2025-04-11 12:12:10,262 - INFO - Datos cargados exitosamente desde CSV. Filas iniciales: 99


In [80]:
                                       
if df_yt_stats is not None:
    logging.info(f"Creando copia del DataFrame para limpieza: '{CLEAN_TABLE_NAME}'")
                                           
    youtube_stats_clean = df_yt_stats.copy()
else:
    logging.error("No se puede iniciar la limpieza porque 'df_yt_stats' no está cargado.")
    raise ValueError("DataFrame 'df_yt_stats' original no disponible para copiar.")


2025-04-11 12:12:10,279 - INFO - Creando copia del DataFrame para limpieza: 'youtube_stats_clean'


In [81]:
                                                     
if youtube_stats_clean is not None:
    logging.info("--- Iniciando Limpieza del DataFrame YouTube Stats ---")
    print(f"Filas antes de la limpieza: {len(youtube_stats_clean)}")
else:
    logging.error("El DataFrame 'youtube_stats_clean' no se pudo crear.")

2025-04-11 12:12:10,299 - INFO - --- Iniciando Limpieza del DataFrame YouTube Stats ---


Filas antes de la limpieza: 99


In [82]:
                                                                                       
logging.info("Rellenando valores NaN en columnas numéricas con 0...")
cols_numericas_a_rellenar = ['subscriber_count', 'view_count', 'video_count', 'total_top10_video_likes']
nulos_rellenados = {}
for col in cols_numericas_a_rellenar:
    if col in youtube_stats_clean.columns:
        nulos_antes = youtube_stats_clean[col].isnull().sum()
        if nulos_antes > 0:
                                                                        
            if col == 'subscriber_count':
                    youtube_stats_clean[col] = youtube_stats_clean[col].replace(-1.0, np.nan)                         
                    nulos_antes = youtube_stats_clean[col].isnull().sum()                   
                                
            youtube_stats_clean[col].fillna(0, inplace=True)
            nulos_rellenados[col] = nulos_antes
            logging.info(f"  - Columna '{col}': {nulos_antes} NaN rellenados con 0.")
        else:
                logging.info(f"  - Columna '{col}': No se encontraron NaN para rellenar.")
    else:
        logging.warning(f"  - Columna '{col}' no encontrada para rellenar.")
if nulos_rellenados:
    logging.info("Relleno de NaN en columnas numéricas completado.")

2025-04-11 12:12:10,329 - INFO - Rellenando valores NaN en columnas numéricas con 0...
2025-04-11 12:12:10,336 - INFO -   - Columna 'subscriber_count': 12 NaN rellenados con 0.
2025-04-11 12:12:10,339 - INFO -   - Columna 'view_count': 12 NaN rellenados con 0.
2025-04-11 12:12:10,344 - INFO -   - Columna 'video_count': 12 NaN rellenados con 0.
2025-04-11 12:12:10,347 - INFO -   - Columna 'total_top10_video_likes': 12 NaN rellenados con 0.
2025-04-11 12:12:10,349 - INFO - Relleno de NaN en columnas numéricas completado.


In [83]:
                                             
logging.info("Convirtiendo columnas numéricas a tipo Integer...")
for col in cols_numericas_a_rellenar:
        if col in youtube_stats_clean.columns:
            try:
                                                                                                     
                                                                
                youtube_stats_clean[col] = youtube_stats_clean[col].astype('int64')
                logging.info(f"  - Columna '{col}' convertida a int64.")
            except Exception as e:
                logging.error(f"  - Error al convertir '{col}' a entero: {e}. Se mantendrá como {youtube_stats_clean[col].dtype}.")
        else:
            logging.warning(f"  - Columna '{col}' no encontrada para convertir tipo.")

2025-04-11 12:12:10,371 - INFO - Convirtiendo columnas numéricas a tipo Integer...
2025-04-11 12:12:10,375 - INFO -   - Columna 'subscriber_count' convertida a int64.
2025-04-11 12:12:10,378 - INFO -   - Columna 'view_count' convertida a int64.
2025-04-11 12:12:10,381 - INFO -   - Columna 'video_count' convertida a int64.
2025-04-11 12:12:10,385 - INFO -   - Columna 'total_top10_video_likes' convertida a int64.


In [84]:
                                 
logging.info("Eliminando espacios en blanco iniciales/finales de columnas de texto...")
cols_texto = ['artist_query', 'channel_id_found', 'channel_title_verified']
cols_stripped = []
for col in cols_texto:
    if col in youtube_stats_clean.columns:
                                                                        
        if pd.api.types.is_string_dtype(youtube_stats_clean[col]) or youtube_stats_clean[col].dtype == 'object':
                                                                                            
                youtube_stats_clean[col].fillna('', inplace=True)
                youtube_stats_clean[col] = youtube_stats_clean[col].astype(str).str.strip()
                cols_stripped.append(col)
        else:
                logging.warning(f"  - La columna '{col}' no es de tipo texto, no se aplica strip.")
if cols_stripped:
    logging.info(f"Espacios eliminados en columnas: {cols_stripped}")

2025-04-11 12:12:10,404 - INFO - Eliminando espacios en blanco iniciales/finales de columnas de texto...
2025-04-11 12:12:10,413 - INFO - Espacios eliminados en columnas: ['artist_query', 'channel_id_found', 'channel_title_verified']


In [85]:
                                                  
logging.info("Convirtiendo columnas de texto a tipo 'string' de Pandas...")
try:
    for col in cols_texto:
            if col in youtube_stats_clean.columns:
                youtube_stats_clean[col] = youtube_stats_clean[col].astype('string')
    logging.info("Columnas de texto convertidas a tipo 'string' de Pandas.")
except Exception as e:
    logging.warning(f"No se pudieron convertir todas las columnas a 'string': {e}")


logging.info("--- Limpieza del DataFrame Finalizada ---")
print(f"Filas después de la limpieza: {len(youtube_stats_clean)}")

2025-04-11 12:12:10,429 - INFO - Convirtiendo columnas de texto a tipo 'string' de Pandas...
2025-04-11 12:12:10,435 - INFO - Columnas de texto convertidas a tipo 'string' de Pandas.
2025-04-11 12:12:10,437 - INFO - --- Limpieza del DataFrame Finalizada ---


Filas después de la limpieza: 99


In [86]:
                                            
logging.info("\n--- Verificación del DataFrame Limpio ('youtube_stats_clean') ---")
print("\nPrimeras filas del DataFrame limpio:")
display(youtube_stats_clean.head())
print("\nInformación del DataFrame limpio (tipos en Pandas):")
youtube_stats_clean.info()
print("\nConteo de nulos por columna en el DataFrame limpio:")
display(youtube_stats_clean.isnull().sum())                                          

2025-04-11 12:12:10,457 - INFO - 
--- Verificación del DataFrame Limpio ('youtube_stats_clean') ---



Primeras filas del DataFrame limpio:


,artist_query,channel_id_found,channel_title_verified,subscriber_count,view_count,video_count,total_top10_video_likes
0,Nalan,UC_zzCBiTkpQwP8lwHgQ7M3Q,Nalan - Topic,19400,53723636,187,163164
1,Grupo Sensação,UCKiMawhTZ5z1S8i1_fezSAw,Grupo Sensação,55800,8161696,24,0
2,Gorillaz;Beck,UCNIV5B_aJnLrKDSnW_MOmcQ,Gorillaz - Topic,43100,1533592212,560,9354631
3,Parcels,UC2as7PrmUgmdZAkMIWNY6EQ,Parcels,306000,180354793,236,745054
4,Klingande,UCOX8OMkI7ULP7K8bfB_HTHA,Klingande,83500,64798541,44,163822



Información del DataFrame limpio (tipos en Pandas):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   artist_query             99 non-null     string
 1   channel_id_found         99 non-null     string
 2   channel_title_verified   99 non-null     string
 3   subscriber_count         99 non-null     int64 
 4   view_count               99 non-null     int64 
 5   video_count              99 non-null     int64 
 6   total_top10_video_likes  99 non-null     int64 
dtypes: int64(4), string(3)
memory usage: 5.5 KB

Conteo de nulos por columna en el DataFrame limpio:


artist_query               0
channel_id_found           0
channel_title_verified     0
subscriber_count           0
view_count                 0
video_count                0
total_top10_video_likes    0
dtype: int64

In [87]:
                                                                       
engine = None                                          
if all([DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME]):
    try:
        logging.info(f"Creando motor SQLAlchemy para la base de datos '{DB_NAME}' (para carga)...")
        db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(db_url)
        logging.info(f"Motor SQLAlchemy para carga creado exitosamente.")
    except Exception as e:
        logging.error(f"Error al crear el motor SQLAlchemy para carga: {e}")
        engine = None                                      
else:
    logging.error("Faltan variables de entorno de DB, no se puede crear engine para carga.")

2025-04-11 12:12:10,510 - INFO - Creando motor SQLAlchemy para la base de datos 'artists' (para carga)...
2025-04-11 12:12:10,582 - INFO - Motor SQLAlchemy para carga creado exitosamente.


In [88]:
                                                                                       
                                                                     
if youtube_stats_clean is not None and engine is not None:

                                                             
    sql_types = {
        'artist_query': Text(),
        'channel_id_found': Text(),                                
        'channel_title_verified': Text(),                          
        'subscriber_count': BigInteger(),                                
        'view_count': BigInteger(),                                      
        'video_count': Integer(),                               
        'total_top10_video_likes': BigInteger()                            
    }

    logging.info(f"Cargando DataFrame limpio a la tabla '{CLEAN_TABLE_NAME}' en chunks de {CHUNK_SIZE}...")
    start_upload_time = time.time()
    try:
        youtube_stats_clean.to_sql(
            CLEAN_TABLE_NAME,
            con=engine,
            if_exists='replace',
            index=False,
            method='multi',
            dtype=sql_types,
            chunksize=CHUNK_SIZE
        )
        end_upload_time = time.time()
        logging.info(f"DataFrame limpio cargado exitosamente en la tabla '{CLEAN_TABLE_NAME}' en {end_upload_time - start_upload_time:.2f} segundos.")

                                                
        logging.info(f"Verificando número de filas en la tabla '{CLEAN_TABLE_NAME}'...")
        with engine.connect() as connection:
            query_count = text(f'SELECT COUNT(*) FROM "{CLEAN_TABLE_NAME}"')
            result = connection.execute(query_count)
            num_db_clean_rows = result.scalar_one()

        logging.info(f"Número de filas en la tabla limpia '{CLEAN_TABLE_NAME}': {num_db_clean_rows}")
        logging.info(f"Número de filas en el DataFrame limpio: {len(youtube_stats_clean)}")

        if len(youtube_stats_clean) == num_db_clean_rows:
            logging.info("¡Verificación de carga final exitosa!")
        else:
            logging.warning("Discrepancia en el número de filas entre el DataFrame limpio y la tabla cargada.")

    except Exception as e:
        logging.error(f"Error al cargar el DataFrame limpio en la base de datos: {e}")
        raise

elif youtube_stats_clean is None:
     logging.error("No se puede cargar la tabla limpia porque el DataFrame 'youtube_stats_clean' no está definido.")
elif engine is None:
     logging.error("No se puede cargar la tabla limpia porque la conexión a la base de datos (engine) no está definida.")

logging.info("--- Proceso de Limpieza y Carga de YouTube Stats Finalizado ---")

2025-04-11 12:12:10,639 - INFO - Cargando DataFrame limpio a la tabla 'youtube_stats_clean' en chunks de 40000...
2025-04-11 12:12:10,742 - INFO - DataFrame limpio cargado exitosamente en la tabla 'youtube_stats_clean' en 0.09 segundos.
2025-04-11 12:12:10,744 - INFO - Verificando número de filas en la tabla 'youtube_stats_clean'...
2025-04-11 12:12:10,747 - INFO - Número de filas en la tabla limpia 'youtube_stats_clean': 99
2025-04-11 12:12:10,748 - INFO - Número de filas en el DataFrame limpio: 99
2025-04-11 12:12:10,749 - INFO - ¡Verificación de carga final exitosa!
2025-04-11 12:12:10,750 - INFO - --- Proceso de Limpieza y Carga de YouTube Stats Finalizado ---
